In [ ]:
import numpy as np

from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

### **Load data and preprocess**

In [ ]:
dataset = load_iris()
x = dataset.data
y = dataset.target

print("shape of input:", x.shape)
print("shape of output:", y.shape)

print("EXAMPLE:")
print("Input:", x[0])
print("Output:", y[0])

Let encode output with one hot encoding

In [ ]:
def encode_with_one_hot_encoding(categorical_values):
    num_of_classes = len(np.unique(categorical_values))
    return np.eye(num_of_classes)[categorical_values]

encoded_y = encode_with_one_hot_encoding(y)
print("examples of encoded y:")
print(encoded_y[:10])

Transpose data

In [ ]:
X = x.T
Y = encoded_y.T

print("input shape:", X.shape)
print("target shape:", Y.shape)

Describe data

In [ ]:
def describe_data(data):
    return f'number of features is {data.shape[0]} and number of datapoints is {data.shape[1]}'

print(f'For input: {describe_data(X)}')
print(f'For output: {describe_data(Y)}')

### **Creating neural network**

In neural network, the total input $x_j$ to neuron j is a linear function of the output

**Weights of layer l:**

$\theta^{(l)} = \begin{bmatrix} 
                \theta_{11}^{(l)} & \theta_{12}^{(l)} & ... & \theta_{1n}^{(l)} \\
                \theta_{21}^{(l)} & \theta_{22}^{(l)} & ... & \theta_{2n}^{(l)} \\
                ... & ... & ... & ... \\
                \theta_{m1}^{(l)} & \theta_{m2}^{(l)} & ... & \theta_{mn}^{(l)} \\
                \end{bmatrix}
$

Where:
 - m is the number of input features
 - n is the number of neurons
 - $\theta_{16}^{(2)}$ is the weight of the connection between the first feature and $6^{th}$ neuron in second layer
 - Shape of layer l 's weights is $m \times n = $ number of features $\times$ number of neurons

Create neural network and implement forward propagation

In [ ]:
def create_neural_network(input_size, output_size, hidden_sizes):
    """
    Parameters:
        - input_size: 
        - output_size: number of neurons of the last layer
        - hidden_sizes: number of neurons of hidden layers
    """
    network_wetghts = []
    layer_sizes = hidden_sizes
    layer_sizes.append(output_size)
    for num_of_neurons in layer_sizes:
        layer_weights = np.random.rand(input_size, num_of_neurons)*2-1  # weights are initialized in range [-1, 1]
        input_size = num_of_neurons
        network_wetghts.append(layer_weights)
    return network_wetghts

In [ ]:
def describe_layer(layer_weights):
    return f'there is {layer_weights.shape[1]} neurons with {layer_weights.shape[0]} inputs each'

neural_network_weights = create_neural_network(X.shape[0], Y.shape[0], [8, 6])
for idx, layer_weights in enumerate(neural_network_weights):
    print(f'In layer {idx} {describe_layer(layer_weights)}')

Now we have a neural network as below image:

Output of layer 1 will be calculated by:

$A^{(1)} = (W^{(1)})^T \times X^{T} $

<p align="center">
  <img src="network_wo_bias.png" alt="simple neural network" width="650"/>
</p>


**But wait** ! Do you think we have forgot something ?

Yes! **Bias**

Our neural network should be represented as:

<p align="center">
  <img src="network.png" alt="neural network" width="650"/>
</p>

Bias is an adjustable, numerical term added to a perceptron’s weighted sum of inputs and weights. Bias allows perceptrons to account for situations where the inputs are insufficient to capture the complexity of the problem. It enables the perceptron to learn better decision boundaries by shifting them in the desired direction.

In [ ]:
def create_neural_network_with_bias(input_size, output_size, hidden_sizes):
    """
    Parameters:
        - input_size: 
        - output_size: number of neurons of the last layer
        - hidden_sizes: number of neurons of hidden layers
    """
    network_weights = []
    network_biases = []
    layer_sizes = hidden_sizes
    layer_sizes.append(output_size)
    for num_of_neurons in layer_sizes:
        layer_weights = np.random.rand(input_size, num_of_neurons)*2-1  # weights are initialized in range [-1, 1]
        layer_bias = np.random.rand(1, num_of_neurons)*2-1  # bias are initialized in range [-1, 1]
        network_weights.append(layer_weights)
        network_biases.append(layer_bias)
        input_size = num_of_neurons
    return network_weights, network_biases

In [ ]:
def describe_bias(layer_bias):
    return f'bias has shape {layer_bias.shape} for {layer_bias.shape[1]} neurons'

neural_network_weights, neural_network_biases = create_neural_network_with_bias(X.shape[0], Y.shape[0], [8, 6])
for idx, layer_weights in enumerate(neural_network_weights):
    print(f'In layer {idx} {describe_layer(layer_weights)} and {describe_bias(neural_network_biases[idx])}')

And **activation function**

Activation function is used to determine the output of neural network like yes or no. It maps the resulting values in between 0 to 1 or -1 to 1 etc. (depending upon the function).

In this example, we are classifying iris flowers. Therefore, sigmoid activation function is used for models to predict the probability because its output is always between 0 and 1. 

Sigmoid activation function:

$\sigma(u) = \frac{1}{1 + e^{-u}}$

Sigmoid derivative is:

$\frac{d}{du} \sigma(u) = \frac{d}{du} \frac{1}{1 + e^{-u}} = -(1 + e^{-u})^{-2} \cdot \frac{d}{du} (1 + e^{-u})
                        = -(1 + e^{-u})^{-2} \cdot \frac{d}{du} (e^{-u})
                        = -(1 + e^{-u})^{-2} \cdot e^{-u} \cdot \frac{d}{du} (-u)
                        = (1 + e^{-u})^{-2} \cdot e^{-u} 
                        = \sigma(u) \cdot (1 - \sigma(u))
$

In [ ]:
def sigmoid(u):
    return 1 / (1 + np.exp(-u))

def sigmoid_derivative(u):
    a = sigmoid(u)
    return a * (1.0 - a)

In [ ]:
# Example to understand sigmoid and its derivative
demo_x = np.linspace(-10, 10, 100, endpoint=True)
demo_y = sigmoid(demo_x)
demo_derivative = sigmoid_derivative(demo_x)
plt.plot(demo_x, demo_y, label='Sigmoid function')
plt.plot(demo_x, demo_derivative, label='Sigmoid derivative')
plt.legend()
plt.show()

### **Forward propagation**

Output of layer 1 will be calculated by:

$A^{(1)} = (W^{(1)})^T \times X^{T} + (b^{(1)})^{T}$

<p align="center">
  <img src="feed_forward_1.png" alt="feed forward" width="650"/>
</p>

Let's do a simplifying trick:

Extend input by adding 1 to every input vector. Bias becomes the weight on this extra input. It can be treated just like other weights. This trick is used to be able to include bias in weights

<p align="center">
  <img src="feed_forward_2.png" alt="simplify feed forward" width="650"/>
</p>

Output of layer 1 will be calculated by:

$A^{(1)} = \sigma ((W^{(1)})^T \times X^{T} + (b^{(1)})^{T}) = \sigma ((W^{(1)} || b^{(1)})^T \times (X || 1)^{T})$ 

Extend input with 1

In [ ]:
def extend_input_with_ones(input_T):
    return np.concatenate((input_T, np.ones(shape=(1, input_T.shape[1]))), axis=0)

Extend weights with bias

In [ ]:
def extend_weights_with_bias(weights, bias):
    return np.concatenate((weights, bias), axis=0)

Feed forward code for neural network

In [ ]:
def feed_forward(input_data, network_weights, network_biases):
    layer_input = extend_input_with_ones(input_data)
    layers_output = []
    for layer_weights, layer_bias in zip(network_weights, network_biases):     # same meaning as for layer in network
        extended_weights = extend_weights_with_bias(layer_weights, layer_bias)  # extended weights include bias
        layer_output = sigmoid(extended_weights.T @ layer_input) # @ denotes matrix multiply (different from * which denotes dot multiply)
        layer_input = extend_input_with_ones(layer_output) # input of next layer = output of current layer extend with ones
        layers_output.append(layer_output)
    return layers_output

**Loss function**

In this example, we use mean square error as loss function

$MSE = \frac{1}{n} \sum_{i=1}^{n} (gt_{i} - pred_{i})^2 
= \frac{1}{n} \sum (Y - \hat{Y})^2 
$

In [ ]:
def calculate_mse(groundtruth, prediction):
    return np.sum((groundtruth - prediction)**2)/len(groundtruth)

In [ ]:
# Feed forward
outputs = feed_forward(X, neural_network_weights, neural_network_biases)
prediction = outputs[-1]

# Describe output of each layer
for idx, layer_output in enumerate(outputs):
    print(f'Layer {idx} output has {describe_data(layer_output)}')

# Calculate MSE
mse  = calculate_mse(Y, prediction)
print(f'MSE = {mse}')

**Take this step slower to understand it.**

![Neural network](network.png)

First, you have input data X has 4 features $\times$ 150 samples.

And a neural network has 3 layers:
 - Layer 1 has 8 neurons
 - Layer 2 has 6 neurons
 - Layer 3 has 3 neurons

As we mentioned above, weights matrix of each layer has shape = number of input features $\times$ number of neurons in that layer. 

Therefore:
 - Layer 1: weights matrix has shape 4 $\times$ 8, bias matrix has shape 1 $\times$ 8
 - Layer 2 weights matrix has shape 8 $\times$ 6, bias matrix has shape 1 $\times$ 6
 - Layer 3 weights matrix has shape 6 $\times$ 3, bias matrix has shape 1 $\times$ 3

Output of each layer is: $\text{weights}^{T} \times \text{layer input}^{T} + \text{bias}^{T}$. Then:
 - Output of layer 1 has shape of (8, 4) $\times$ (4, 150) + broadcast(8, 1)= (8, 150)
 - Output of layer 2 has shape of (6, 8) $\times$ (8, 150)  + broadcast(6, 1)= (6, 150)
 - Output of layer 3 has shape of (3, 6) $\times$ (6, 150) + broadcast(3, 1)= (3, 150). This is shape of the prediction

In [ ]:
prediction.shape

Decode the output 

In [ ]:
np.argmax(prediction, axis=0)

### **Backward propagation**

First, calculate the gradient of loss function MSE with respect to weights.

![computation graph and MSE loss](MSE_loss.png)

As $MSE =  \frac{1}{n} \sum (Y - \hat{Y})^2 $

Let's consider: $L = (Y - \hat{Y})^2$

Then:
$\frac{dL}{d\hat{Y}} = d\frac{(Y - \hat{Y})^2}{d\hat{Y}} = 2(Y - \hat{Y})d \frac{Y - \hat{Y}}{d\hat{Y}} = -2(Y - \hat{Y}) = 2(\hat{Y}-Y)$

But: $\hat{Y} = \sigma(W^{(3)} \times A^{(2)} + b^{(3)})$
then: 
$
\frac{d\hat{Y}}{dW^{(3)}} = d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{dW^{(3)}} 
$

Gradient of L respect to **weights**:

$\frac{dL}{dW^{(3)}} = \frac{dL}{d\hat{Y}}.\frac{d\hat{Y}}{dW^{(3)}} 
= 2(\hat{Y}-Y) d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{dW^{(3)}} 
= 2(\hat{Y}-Y) A^{(2)} d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{d(W^{(3)} \times A^{(2)} + b^{(3)})}
$

Additionally:

$\frac{dL}{dW^{(2)}} = \frac{dL}{d\hat{Y}}.\frac{d\hat{Y}}{dA^{(2)}}.\frac{dA^{(2)}}{dW^{(2)}}
= 2(\hat{Y}-Y) 
d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{dA^{(2)}} 
d\frac{\sigma(W^{(2)} \times A^{(1)} + b^{(2)})}{dW^{(2)}} 
= 2(\hat{Y}-Y) W^{(3)} 
d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{d(W^{(3)} \times A^{(2)} + b^{(3)})} A^{(1)} 
d\frac{\sigma(W^{(2)} \times A^{(1)} + b^{(2)})}{d(W^{(2)} \times A^{(1)} + b^{(2)})} 
$

$\frac{dL}{dW^{(1)}} = \frac{dL}{d\hat{Y}}.\frac{d\hat{Y}}{dA^{(2)}}.\frac{dA^{(2)}}{dA^{(1)}}.\frac{dA^{(1)}}{dW^{(1)}}
= 2(\hat{Y}-Y) d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{dA^{(2)}} 
d\frac{\sigma(W^{(2)} \times A^{(1)} + b^{(2)})}{dA^{(1)}} 
d\frac{\sigma(W^{(1)} \times X + b^{(1)})}{dW^{(1)}} 
= 2(\hat{Y}-Y) W^{(3)} 
d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{d(W^{(3)} \times A^{(2)} + b^{(3)})} W^{(2)} 
d\frac{\sigma(W^{(2)} \times A^{(1)} + b^{(2)})}{d(W^{(2)} \times A^{(1)} + b^{(2)})} X 
d\frac{\sigma(W^{(1)} \times X + b^{(1)})}{d(W^{(1)} \times X + b^{(1)})} 
$


Gradient of L respect to **bias**:

$\frac{dL}{db^{(3)}} = \frac{dL}{d\hat{Y}}.\frac{d\hat{Y}}{db^{(3)}} 
= 2(\hat{Y}-Y) d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{db^{(3)}} 
= 2(\hat{Y}-Y) d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{d(W^{(3)} \times A^{(2)} + b^{(3)})}
$

Additionally:

$\frac{dL}{db^{(2)}} = \frac{dL}{d\hat{Y}}.\frac{d\hat{Y}}{dA^{(2)}}.\frac{dA^{(2)}}{db^{(2)}}
= 2(\hat{Y}-Y) 
d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{dA^{(2)}} 
d\frac{\sigma(W^{(2)} \times A^{(1)} + b^{(2)})}{db^{(2)}} 
= 2(\hat{Y}-Y) W^{(3)} 
d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{d(W^{(3)} \times A^{(2)} + b^{(3)})} 
d\frac{\sigma(W^{(2)} \times A^{(1)} + b^{(2)})}{d(W^{(2)} \times A^{(1)} + b^{(2)})} 
$

$\frac{dL}{db^{(1)}} = \frac{dL}{d\hat{Y}}.\frac{d\hat{Y}}{dA^{(2)}}.\frac{dA^{(2)}}{dA^{(1)}}.\frac{dA^{(1)}}{dW^{(1)}}
= 2(\hat{Y}-Y) d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{dA^{(2)}} 
d\frac{\sigma(W^{(2)} \times A^{(1)} + b^{(2)})}{dA^{(1)}} 
d\frac{\sigma(W^{(1)} \times X + b^{(1)})}{dW^{(1)}} 
= 2(\hat{Y}-Y) W^{(3)} 
d\frac{\sigma(W^{(3)} \times A^{(2)} + b^{(3)})}{d(W^{(3)} \times A^{(2)} + b^{(3)})} W^{(2)} 
d\frac{\sigma(W^{(2)} \times A^{(1)} + b^{(2)})}{d(W^{(2)} \times A^{(1)} + b^{(2)})} 
d\frac{\sigma(W^{(1)} \times X + b^{(1)})}{d(W^{(1)} \times X + b^{(1)})} 
$

In [ ]:
def backpropagate(network_weights, network_biases, input, outputs, target):
    ''' 
    Parameters:
        - network: [W1, W2, W3]
        - outputs: [a1, a2, y_hat]
        - target: y
    Return:
        - Gradient of loss function respect to model weights & model biases
    '''

    gradients_weights = []
    gradients_biases = []

    # Create A = [X, a1, a2, y_hat]
    A = [input]
    for output in outputs:
        A.append(output)
    
    # delta = 2 * (y_hat - y) * sigmoid_derivative(y_hat)
    delta = 2 * (outputs[-1] - target) * sigmoid_derivative(A[-1])
    
    # loop over the layers in the network in reverse order
    for layer_idx in reversed(range(0, len(network_weights)-1)):
        print(f'back propagate in layer {layer_idx}')
        gradients_weights.append(np.dot(A[layer_idx].T, delta))    
        gradients_biases.append((np.sum(delta, 0)).reshape(-1,1))
        delta = (delta * sigmoid_derivative(A[layer_idx])).dot(gradients_weights[layer_idx].T)    

    return zip(reversed(gradients_weights), reversed(gradients_biases))


In [ ]:
gradients_weights, gradients_biases = backpropagate(neural_network_weights, neural_network_biases, X, outputs, Y)
for idx, gw in enumerate(gradients_weights):
    print(f'Gradient weight of {idx} layer: {describe_data(gw)}')
for idx, gb in enumerate(gradients_biases):
    print(f'Gradient weight of {idx} layer: {describe_data(gb)}')

Then update weights and biases by gradient descent

In [ ]:
def gradient_descent(network_weights, network_biases, gradients_weights, gradients_biases , lr):
    for i in range(len(network_weights)):
        network_weights[i] = network_weights[i] - lr * gradients_weights[i]
        network_biases[i] = network_biases[i] - lr * gradients_biases[i]
    return network_weights, network_biases

Then adjust weights by weights changes

In [ ]:
def adjust_weights(network, changes):
    new_network = []
    for weights, change in zip(network, changes):
        new_weights = weights - change
        new_network.append(new_weights)
    return new_network

In [ ]:
neural_network = adjust_weights(neural_network, changes)

# Describe each layer
for idx, layer in enumerate(neural_network):
    print(f'In layer {idx} {describe_layer(layer)}')

# Feed forward
prediction = feed_forward(X, neural_network)[-1]

# Calculate MSE
mse  = calculate_mse(Y, prediction)
print(f'MSE = {mse}')

### Congratulation! Here you come

## Train

In [ ]:
def train(network, input, target, lr, epochs):
    mse_history = []
    for epoch in epochs:
        outputs = feed_forward(input, network)
        mse_history.append(calculate_mse(outputs[-1], target))
        gradients = backpropagate(network, outputs, target)
        changes = calculate_weights_changes(network, input, outputs, gradients, lr)
        network = adjust_weights(network, changes)
    mse_history.append(calculate_mse(target, outputs[-1]))
    return network, mse_history

### Sources:

1. [https://nttuan8.com/bai-4-backpropagation/](https://nttuan8.com/bai-4-backpropagation/)
2. [https://youtu.be/Cwr0H0kAMtM?si=eG_B1AgeywbuZu0N](https://youtu.be/Cwr0H0kAMtM?si=eG_B1AgeywbuZu0N)
3. [http://cs231n.stanford.edu/slides/2023/lecture_2.pdf](http://cs231n.stanford.edu/slides/2023/lecture_2.pdf)
4. [http://cs231n.stanford.edu/schedule.html](http://cs231n.stanford.edu/schedule.html)
5. [http://cs231n.stanford.edu/handouts/linear-backprop.pdf](http://cs231n.stanford.edu/handouts/linear-backprop.pdf)
6. [https://pyimagesearch.com/2021/05/06/backpropagation-from-scratch-with-python/](https://pyimagesearch.com/2021/05/06/backpropagation-from-scratch-with-python/)
7. [http://neuralnetworksanddeeplearning.com/chap2.html](http://neuralnetworksanddeeplearning.com/chap2.html)
8. [https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/](https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/)
9. [https://dustinstansbury.github.io/theclevermachine/a-gentle-introduction-to-neural-networks](https://dustinstansbury.github.io/theclevermachine/a-gentle-introduction-to-neural-networks)
10. [https://dustinstansbury.github.io/theclevermachine/derivation-backpropagation](https://dustinstansbury.github.io/theclevermachine/derivation-backpropagation)